In [46]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
from patsy import dmatrix
# Import Necessary Libraries

In [47]:
df = pd.read_csv("processed_df.csv", index_col = 0)
mrns = pd.read_csv("mrns.csv")
mrns = mrns[['DATE_CIRRHOSIS', 'MRN', 'Gender', 'Age']]

df['Date'] = pd.to_datetime(df['Date'], format='mixed', dayfirst=True)
mrns['DATE_CIRRHOSIS'] = pd.to_datetime(mrns['DATE_CIRRHOSIS'], format='mixed', dayfirst=True)
df
# Read two csv files, df and mrns
# df is a longitudinal data with columns: MRN, Date of Check-up, Tac Dose Level, Tac Trough level, Bilirubin,
# Creatinine, Tac Dose-To-Trough Ratio, Age at the time of checkup, 
# and the number of days relative to the first check-up (For example, for MRN 141619, checkup at 2007-04-09 has value 0,
# since it is the first check-up, while 2007-06-20 has value 72, since it is 72 days after the first check-up)

,MRN,Date,Dose,Level,Bili,Creat,Meld,Ratio,Age,Time
0,141619,2007-04-09,10.0,13.3,5.0,93.0,7.0,1.330,58.633128,0
1,141619,2007-06-20,10.0,17.0,8.0,85.0,7.0,1.700,58.830253,72
2,141619,2007-07-26,6.0,19.2,10.0,96.0,8.0,3.200,58.928816,108
3,498916,2005-05-26,4.0,10.5,17.0,101.0,10.0,2.625,46.932923,0
4,498916,2005-06-02,4.0,11.6,23.0,120.0,13.0,2.900,46.952088,7
...,...,...,...,...,...,...,...,...,...,...
6535,6381914,2009-11-02,4.0,5.7,25.0,76.0,10.0,1.425,61.726215,287
6536,6381914,2009-11-11,4.0,8.1,19.0,68.0,9.0,2.025,61.750856,296
6537,6381914,2009-12-11,4.0,7.3,40.0,66.0,12.0,1.825,61.832991,326
6538,6381914,2010-01-14,4.0,9.0,31.0,70.0,11.0,2.250,61.926078,360


In [48]:
# dataframe mrns have information about date of cirrhosis, gender and age at the diagnosis of cirrhosis for 
# each individual patients (mrns)
mrns

,DATE_CIRRHOSIS,MRN,Gender,Age
0,2020-02-24,4305337,Female,60.0
1,2020-01-25,2422830,NaN,NaN
2,2020-01-24,3813561,Male,75.0
3,2020-01-21,3647973,Male,68.0
4,2020-01-17,576112,NaN,NaN
...,...,...,...,...
448,1999-06-03,1160986,NaN,NaN
449,1999-01-27,2024219,NaN,NaN
450,1999-01-19,2387271,NaN,NaN
451,1999-01-12,1125534,NaN,NaN


## Defining Spline Terms
Since the mixedlm from statsmodels library is a Linear Mixed Effect Model, I tried to use cubic splines to apply nonlinearity. I thought instead of simply taking polynomial terms of Times, taking spline terms would help me to capture more complex relationship

In [49]:
spline_terms = dmatrix("bs(Time, degree=3, knots=(100,), include_intercept=False)", df, return_type='dataframe')
spline_terms.columns = [f'spline_{i}' for i in range(spline_terms.shape[1])]
df = pd.concat([df, spline_terms], axis=1)
# define the spline terms, concatenate those terms with original df

In [89]:
from sklearn.preprocessing import StandardScaler
model_df_columns = df.columns[4:]
model_df = df.iloc[:, 4:]
scaler = StandardScaler()
model_df = scaler.fit_transform(model_df)
model_df = pd.DataFrame(columns = model_df_columns, data = model_df)
model_df = pd.concat([model_df, df['MRN'].reset_index(drop = True)], axis = 1)
model_df = model_df.dropna()
model_df
# Drop Date (since we have another variable Time), Tac Dose and Tac trough level
# standardize all the columns (Bilirubin, Creatinine, Meld, Ratio, Age, Time and splines)

,Bili,Creat,Meld,Ratio,Age,Time,spline_0,spline_1,spline_2,spline_3,spline_4,MRN
0,-0.513067,-0.205467,-0.739657,-0.735696,0.485304,-0.995817,0.0,-2.217302,-1.565599,-0.725838,-0.295201,141619
1,-0.424059,-0.346427,-0.739657,-0.580551,0.501660,-0.923212,0.0,1.254755,-1.439538,-0.724966,-0.295201,141619
2,-0.364719,-0.152607,-0.493105,0.048419,0.509838,-0.886910,0.0,1.272951,-1.328620,-0.722896,-0.295201,141619
3,-0.157032,-0.064506,0.000000,-0.192686,-0.485495,-0.995817,0.0,-2.217302,-1.565599,-0.725838,-0.295201,498916
4,0.020985,0.270274,0.739657,-0.077375,-0.483905,-0.988758,0.0,-1.509803,-1.564053,-0.725838,-0.295201,498916
...,...,...,...,...,...,...,...,...,...,...,...,...
6535,0.080324,-0.505007,0.000000,-0.695862,0.741946,-0.706407,0.0,0.972615,-0.804756,-0.686641,-0.294810,6381914
6536,-0.097693,-0.645967,-0.246552,-0.444274,0.743990,-0.697331,0.0,0.957988,-0.780133,-0.683709,-0.294751,6381914
6537,0.525368,-0.681207,0.493105,-0.528136,0.750805,-0.667079,0.0,0.909552,-0.699221,-0.673201,-0.294511,6381914
6538,0.258342,-0.610727,0.246552,-0.349929,0.758529,-0.632793,0.0,0.855256,-0.609675,-0.659941,-0.294151,6381914


# Attempt 1: Models with both Men / Women
**Input**: Bili, Creat, Meld, Age, Spline 1, Spline 2, Spline 3, Spline 4
**Output**: Ratio
- I removed spline 0 because spline 0 turned out to be all 0, and it created an error when I ran mixed effect model

In [90]:
model_a1_1= smf.mixedlm(
    "Ratio ~ spline_1 + spline_2 + spline_3 + spline_4 + Bili + Creat + Meld + Age",
    model_df, 
    groups=model_df["MRN"]
)
result_a1_1 = model_a1_1.fit(reml = False)
print(result_a1_1.summary())
# reml was set to False to calculate the AIC and BIC

         Mixed Linear Model Regression Results
Model:            MixedLM Dependent Variable: Ratio     
No. Observations: 6259    Method:             ML        
No. Groups:       143     Scale:              0.4694    
Min. group size:  3       Log-Likelihood:     -6773.2963
Max. group size:  160     Converged:          Yes       
Mean group size:  43.8                                  
--------------------------------------------------------
              Coef.  Std.Err.   z    P>|z| [0.025 0.975]
--------------------------------------------------------
Intercept      0.036    0.061  0.597 0.551 -0.082  0.155
spline_1      -0.005    0.015 -0.325 0.745 -0.034  0.024
spline_2       0.107    0.015  6.980 0.000  0.077  0.137
spline_3       0.034    0.021  1.601 0.109 -0.008  0.076
spline_4       0.041    0.015  2.699 0.007  0.011  0.071
Bili          -0.010    0.014 -0.735 0.462 -0.038  0.017
Creat         -0.025    0.017 -1.472 0.141 -0.059  0.008
Meld           0.136    0.016  8.575 0.00

In [103]:
# Fit the mixed-effects model only using spline 2, which has the lowest p-value of 0.000
model_a1_2 = smf.mixedlm(
    "Ratio ~  spline_2 + Bili + Creat + Meld + Age",
    df1, 
    groups=df1["MRN"]
)
result_a1_2 = model_a1_2.fit(reml = False)

# Print the summary of the model
print(result_a1_2.summary())


         Mixed Linear Model Regression Results
Model:            MixedLM Dependent Variable: Ratio     
No. Observations: 6264    Method:             ML        
No. Groups:       143     Scale:              0.4586    
Min. group size:  3       Log-Likelihood:     -6709.7130
Max. group size:  160     Converged:          Yes       
Mean group size:  43.8                                  
--------------------------------------------------------
              Coef.  Std.Err.   z    P>|z| [0.025 0.975]
--------------------------------------------------------
Intercept      0.022    0.062  0.351 0.726 -0.099  0.142
spline_2       0.106    0.012  8.928 0.000  0.082  0.129
Bili          -0.006    0.014 -0.421 0.673 -0.034  0.022
Creat         -0.031    0.017 -1.780 0.075 -0.064  0.003
Meld           0.135    0.016  8.504 0.000  0.104  0.166
Age            0.229    0.045  5.112 0.000  0.141  0.317
Group Var      0.523    0.099                           



# Attempt 2: Men and Women were separated into Two Groups

In [94]:
men_mrns = mrns[mrns['Gender'] == 'Male']['MRN'].values
women_mrns = mrns[mrns['Gender'] == 'Female']['MRN'].values
men = model_df[model_df['MRN'].isin(men_mrns)].reset_index(drop = True)
women = model_df[model_df['MRN'].isin(women_mrns)].reset_index(drop = True)

In [95]:
model_men_1 = smf.mixedlm(
    "Ratio ~ spline_1 + spline_2 + spline_3 + spline_4 + Bili + Creat + Meld + Age",
    men, 
    groups=men["MRN"]
)
result_men_1 = model_men_1.fit(reml = False)
print(result_men_1.summary())

         Mixed Linear Model Regression Results
Model:            MixedLM Dependent Variable: Ratio     
No. Observations: 3436    Method:             ML        
No. Groups:       80      Scale:              0.4258    
Min. group size:  3       Log-Likelihood:     -3554.9410
Max. group size:  131     Converged:          Yes       
Mean group size:  43.0                                  
--------------------------------------------------------
              Coef.  Std.Err.   z    P>|z| [0.025 0.975]
--------------------------------------------------------
Intercept      0.023    0.080  0.293 0.770 -0.133  0.180
spline_1      -0.001    0.019 -0.075 0.940 -0.039  0.036
spline_2       0.116    0.024  4.767 0.000  0.068  0.164
spline_3      -0.031    0.047 -0.669 0.504 -0.123  0.060
spline_4       0.194    0.053  3.690 0.000  0.091  0.297
Bili          -0.038    0.019 -2.017 0.044 -0.075 -0.001
Creat         -0.076    0.029 -2.661 0.008 -0.132 -0.020
Meld           0.107    0.026  4.107 0.00

In [97]:
model_women_1 = smf.mixedlm(
    "Ratio ~ spline_1 + spline_2 + spline_3 + spline_4 + Bili + Creat + Meld + Age",
    women, 
    groups=women["MRN"]
)
result_women_1 = model_women_1.fit(reml = False)
print(result_women_1.summary())

         Mixed Linear Model Regression Results
Model:            MixedLM Dependent Variable: Ratio     
No. Observations: 2823    Method:             ML        
No. Groups:       63      Scale:              0.5104    
Min. group size:  5       Log-Likelihood:     -3167.6606
Max. group size:  160     Converged:          Yes       
Mean group size:  44.8                                  
--------------------------------------------------------
              Coef.  Std.Err.   z    P>|z| [0.025 0.975]
--------------------------------------------------------
Intercept      0.104    0.092  1.124 0.261 -0.077  0.284
spline_1      -0.025    0.023 -1.049 0.294 -0.071  0.021
spline_2       0.139    0.023  6.127 0.000  0.095  0.184
spline_3      -0.009    0.030 -0.305 0.760 -0.067  0.049
spline_4       0.028    0.017  1.638 0.101 -0.006  0.062
Bili           0.077    0.023  3.360 0.001  0.032  0.122
Creat          0.009    0.022  0.383 0.702 -0.035  0.053
Meld           0.155    0.021  7.533 0.00

In [98]:
model_men_2 = smf.mixedlm(
    "Ratio ~ spline_2 + Bili + Creat + Meld + Age",
    men, 
    groups=men["MRN"]
)
result_men_2 = model_men_2.fit(reml = False)
print(result_men_2.summary())

         Mixed Linear Model Regression Results
Model:            MixedLM Dependent Variable: Ratio     
No. Observations: 3436    Method:             ML        
No. Groups:       80      Scale:              0.4279    
Min. group size:  3       Log-Likelihood:     -3568.7081
Max. group size:  131     Converged:          Yes       
Mean group size:  43.0                                  
--------------------------------------------------------
              Coef.  Std.Err.   z    P>|z| [0.025 0.975]
--------------------------------------------------------
Intercept     -0.044    0.085 -0.514 0.607 -0.210  0.123
spline_2       0.075    0.018  4.242 0.000  0.041  0.110
Bili          -0.034    0.019 -1.826 0.068 -0.071  0.003
Creat         -0.090    0.029 -3.154 0.002 -0.146 -0.034
Meld           0.110    0.026  4.229 0.000  0.059  0.161
Age            0.265    0.079  3.336 0.001  0.109  0.420
Group Var      0.549    0.148                           



In [99]:
model_women_2 = smf.mixedlm(
    "Ratio ~ spline_2 + Bili + Creat + Meld + Age",
    women, 
    groups=women["MRN"]
)
result_women_2 = model_women_2.fit(reml = False)
print(result_women_2.summary())

         Mixed Linear Model Regression Results
Model:            MixedLM Dependent Variable: Ratio     
No. Observations: 2823    Method:             ML        
No. Groups:       63      Scale:              0.5112    
Min. group size:  5       Log-Likelihood:     -3170.5022
Max. group size:  160     Converged:          Yes       
Mean group size:  44.8                                  
---------------------------------------------------------
            Coef.  Std.Err.    z    P>|z|  [0.025  0.975]
---------------------------------------------------------
Intercept   0.125     0.093  1.350  0.177  -0.057   0.307
spline_2    0.132     0.017  7.586  0.000   0.098   0.166
Bili        0.079     0.023  3.475  0.001   0.035   0.124
Creat       0.006     0.022  0.259  0.796  -0.038   0.049
Meld        0.153     0.020  7.493  0.000   0.113   0.194
Age         0.235     0.056  4.191  0.000   0.125   0.345
Group Var   0.508     0.135                              



# AIC and BIC Comparison

In [107]:
labels = ["Both Men / Women, All Splines", "Both Men / Women, Spline 2", "Only Men, All Splines", "Only Men, Spline 2", "Only Women, All Splines", "Only Women, Spline 2"]
aics = [result_a1_1.aic, result_a1_2.aic, result_men_1.aic, result_men_2.aic, result_women_1.aic, result_women_2.aic]
bics = [result_a1_1.bic, result_a1_2.bic, result_men_1.bic, result_men_2.bic, result_women_1.bic, result_women_2.bic]
result_df = pd.DataFrame({
    "AIC": aics,
    "BIC": bics
})
result_df.index = labels
result_df

,AIC,BIC
"Both Men / Women, All Splines",13568.592676,13642.752208
"Both Men / Women, Spline 2",13435.425985,13489.366579
"Only Men, All Splines",7131.882048,7199.444744
"Only Men, Spline 2",7153.416208,7202.552714
"Only Women, All Splines",6357.321228,6422.722338
"Only Women, Spline 2",6357.004327,6404.568770
